## Create Text_Dict dictionary
This file will convert the raw text files for the different text files into a dictinary where each element is {"scene","text"} for example Text_Dict["much_ado_about_nothing_act2scene2"]="Enter Don John and Borach...". This dictinary will later be combined with a dictionary that also contains the summarization of the article.

In [5]:
#Set up
raw_text_location = "../data/text/stories/"

#Importing raw text into strings
def read_raw_text(file_name):
    with open(raw_text_location+file_name, "r") as f:
        contents = f.read()
    return contents

In [16]:
#makes an array with all the text file names
text_files_list=[
    "A_Midsummer_Nights_Dream",
    "Alls_Well_That_Ends_Well",
    "Antony_and_Cleopatra",
    "As_You_Like_It",
    "Coriolanus",
    "Cymbeline",
    "Hamlet",
    "Julius_Caesar",
    "King_Henry_IV_Part_1",
    "King_Henry_IV_Part_1",
    "King_Henry_IV_Part_2",
    "King_Henry_V",
    "King_Henry_VI_Part_2",
    "King_Henry_VI_Part_3",
    "King_Henry_VIII",
    "King_John",
    "King_Lear",
    "King_Richard_II",
    "King_Richard_III",
    "Loves_Labours_Lost",
    "MacBeth",
    "Much_Ado_About_Nothing",
    "Measure_for_Measure",
    "Othello",
    "Pericles_Prince_of_Tyre",
    "Romeo_and_Juliet",
    "The_Comedy_of_Errors",
    "The_Merchent_of_Venice",
    "The_Merry_Wives_of_Windsor",
    "The_Taming_Of_The_Shrew",
    "The_Tempest",
    "The_Two_Gentlemen_of_Verona",
    "The_Two_Noble_Kinsmen",
    "The_Winters_Tale",
    "Timon_of_Athens",
    "Timon_of_Athens",
    "Titus_Andronicus",
    "Titus_Andronicus",
    "Troilus_and_Cressida",
    "Twelfth_Night"
]
print(len(text_files_list))

40


In [6]:
#import raw text and store in dictionary: hamlet_raw_text_files
#each element in the dictionary is (name, text)
hamlet_raw_text_files={}
for e in text_files_list:
    hamlet_raw_text_files[e] = read_raw_text(e+".txt") 

In [7]:
#takes in a string of text (a Shakespeor from Guttentberg) and returns a dictionary with {scene_name: text}
#For example return_var["much_ado_about_nothing_act2scene2"]="Enter Don John and Borach..."]
#Input must be in traditional format of including scene names at the top in a table of contents
def scene_breakout_scenes(raw_text_file, story_name):
    
    #stores scene name in list with each element tuple of structure 
    # ("Act III", "Scene IV.", "Another room in the Castle")
    contents_location=raw_text_file.find("Contents")

    #remove text prior to contents
    raw_text_file=raw_text_file[contents_location:]

    current_act="ACT 0"
    current_scene="SCENE 0."

    scene_name_list=[]

    #loop through contents lines storing scene names in scene_name_list
    #removes each line from raw_text_file after parsed
    while(raw_text_file.find("\n\n\n\n")>0):        
        new_line_loc = raw_text_file.find("\n")
        current_line=raw_text_file[0:new_line_loc]
        raw_text_file=raw_text_file[new_line_loc+1:]

        if current_line[:5]==" ACT " or current_line[:4]=="ACT ":
            start_loc = (1 if current_line[0]==" " else 0) #deals with that sometimes there is a space before the "act"
            current_act=current_line[start_loc:]
            current_scene=0

        elif current_line[:7]==" Scene " or current_line[:6]=="Scene ":
            dot_location = current_line.find(".")
            start_loc = (1 if current_line[:7]==" Scene " else 0) #deals with that sometimes there is a space before the "scene"
            current_scene=current_line[start_loc:dot_location]
            scene_name=current_act+"-"+current_scene+"-"+current_line[dot_location+2:]
            scene_name_list.append(scene_name)
            #scene_name now is a string that looks like ACT II-Scene I-A room in Polonius’s house
            
    
    scene_start_loc_list=[]

    #poplulate scene_start_loc_list with where each of the scene starts 
    for i in range(len(scene_name_list)):
        first_dash=scene_name_list[i].find("-")
        second_dash=scene_name_list[i][first_dash+1:].find("-")+first_dash+1
        
        #find the next scene in the text
        scene_name = scene_name_list[i][first_dash+1:second_dash].upper()+". "+scene_name_list[i][second_dash+1:]
        if i==0:
            scene_start_loc = raw_text_file.find(scene_name)
        else:
            last_start_loc=scene_start_loc_list[i-1]
            scene_start_loc = raw_text_file[last_start_loc:].find(scene_name)+last_start_loc
        scene_start_loc_list.append(scene_start_loc)

    #add end of book to scene_start_loc_list
    scene_start_loc_list.append(raw_text_file.find("*** END OF THE PROJECT GUTENBERG"))

    return_dict={} #will take form {scene_name: text}
    for i in range(len(scene_name_list)):
        start_loc = scene_start_loc_list[i] #start location of scene inclcuding scene title
        start_loc_no_title = raw_text_file.find("\n",start_loc)

        end_loc = scene_start_loc_list[i+1]
        
        scene_text=raw_text_file[start_loc_no_title:end_loc]
        return_dict[story_name+"-"+scene_name_list[i]] = scene_text

    return return_dict




In [8]:
#Makes a dictionary, all_scenes, for each of the scenes
#Dictionary is in from {scene name: scene text}
#Scene name is in format as follows: Title-ACT N-Scene N-Scene Name
    #Ns above are expressed as roman numerals

all_scenes={}

for text_name,text_contents in hamlet_raw_text_files.items():
    all_scenes.update(scene_breakout_scenes(text_contents, text_name))


#At this point all_scenes is a list where each element is a dictionary

In [9]:
#gets count of total scenes
count=0
print(len(all_scenes))

701


In [ ]:
#Looks at scene count for each story
scenes= list(all_scenes.keys())
stories={}
for each in scenes:
    story = each.split("-")[0]
    if story in list(stories.keys()):
        stories[story]+=1
    else:
        stories[story]=0

for story_name, count in stories.items():
    print(story_name,": ",count,sep="")

A_Midsummer_Nights_Dream: 8
Alls_Well_That_Ends_Well: 22
Antony_and_Cleopatra: 41
As_You_Like_It: 21
Coriolanus: 28
Cymbeline: 28
Hamlet: 19
Julius_Caesar: 17
King_Henry_IV_Part_1: 18
King_Henry_IV_Part_2: 18
King_Henry_V: 22
King_Henry_VI_Part_2: 3
King_Henry_VI_Part_3: 27
King_Henry_VIII: 15
King_John: 15
King_Lear: 25
King_Richard_II: 18
King_Richard_III: 24
Loves_Labours_Lost: 8
Much_Ado_About_Nothing: 16
Measure_for_Measure: 16
Othello: 14
Pericles_Prince_of_Tyre: 21
Romeo_and_Juliet: 23
The_Comedy_of_Errors: 10
The_Merchent_of_Venice: 19
The_Merry_Wives_of_Windsor: 22
The_Taming_Of_The_Shrew: 13
The_Tempest: 8
The_Two_Gentlemen_of_Verona: 19
The_Two_Noble_Kinsmen: 23
The_Winters_Tale: 14
Timon_of_Athens: 17
Titus_Andronicus: 13
Troilus_and_Cressida: 23
Twelfth_Night: 17


In [10]:
#Moves dictionary so it can be accessed by other Jupyter notebook files
%store all_scenes

Stored 'all_scenes' (dict)
